## 0. Run this cell at start-up.

In [1]:
import pandas as pd
import ipywidgets as widgets

from nltk.sentiment import SentimentIntensityAnalyzer
import nltk.data

nltk.download('punkt')
nltk.download('vader_lexicon')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
pass

[nltk_data] Downloading package punkt to /Users/thomasj/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/thomasj/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## 1. For a given text, get the average sentiment and sentiment by sentence. 
*re-run this cell each time you want to get a new result*

In [23]:
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from functools import lru_cache
from IPython.display import clear_output
from IPython.display import display

sia = SentimentIntensityAnalyzer()

lru_cache(None)
def sentiment(sentence):
    return sia.polarity_scores(sentence)

def tokenize_sentiment(txt):

    sents = tokenizer.tokenize(txt)
    sentiments = []
    for s in tqdm(sents):
        sentiments.append(sentiment(s))
    return sents, sentiments

output = widgets.Output()

def sentiment_plot(state):
    if text.value=='Paste contents of .txt here.':
        return('Please enter text for all fields')
    
    with output:
        clear_output()
        sents, sentiments = tokenize_sentiment(text.value)
        
        plt.figure(figsize=(12,5))
        ax =  plt.gca()

        df = pd.DataFrame(sentiments)

        print('{} sentiment\n\n'.format(title.value))
        out = df.mean()
        print('sentiment summary scores (average)\n')
        print(out)

        if len(df)>1000:
            smooth = 250
        else:
            smooth = int(len(df)/10)

        ts = (df['compound']).rolling(smooth).mean()
        ax = ts.plot(title='sentiment', ax = ax)
        ax.set_xlabel("narrative time")
        plt.show(plt.gcf())
    return 
    
title = widgets.Text(description='Title:', value='Title')

text = widgets.Textarea(
            value='Paste contents of .txt here.',
            placeholder='Type something',
            description='String:',
            disabled=False,
            layout={'width': '98%', 'height':'250px'}
        )

button = widgets.Button(
    description='Run',
    disabled=False,
    button_style='',
    tooltip='Run',
    icon='', layout={'pos': 'center'}
)

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='flex-start',
                width='50%')

button.on_click(sentiment_plot)
box = widgets.VBox([title, text, button], titles =('Title', 'Text'), layout=box_layout)
display(box, output)


Output()

## 2. For the text entered above, identify plot sections relevant to sentiment graph.

In [12]:
from ipywidgets import interact

@interact(line='', window=(10,100,10))
def print_text(line, window=10):
    try:
        line = int(line)
    except:
        return('Please enter a number.')
    
    sents, sentiments = tokenize_sentiment(text.value)
    df = pd.DataFrame(sentiments)
    
    if len(df)>1000:
        smooth = 250
    else:
        smooth = int(len(df)/10)
        
    ts = (df['pos']-df['neg']).rolling(smooth).mean()
    
    out = []
    for result in zip(list(df[line-window:line+window].index), 
                      sents[line-window:line+window],
                      ts[line-window:line+window]):
        out.append(str((round(result[2],3), result[0], result[1])))
    out = '\n'.join(out)
    print(out)

interactive(children=(Text(value='', description='line'), IntSlider(value=10, description='window', min=10, st…